## Imports

In [1]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from tqdm import tqdm

import keras
from keras import layers
from keras.utils import to_categorical
from keras.models import Model, load_model
#from keras.initializers import glorot_uniform
from keras.layers import Input, Dropout, Add, Dense
from keras.layers import AveragePooling2D, Reshape, Activation
from keras.layers import BatchNormalization, Flatten, Conv1D

import tensorflow as tf
from tensorflow.keras.applications import ResNet50

from sklearn.utils import shuffle
from PIL import Image

import utils

RANDOM_SEED = 1337
SAMPLE_RATE = 32000
SIGNAL_LENGTH = 5 # seconds
SPEC_SHAPE = (48, 128) # height x width
INPUT_SHAPE = SPEC_SHAPE + (3,)
FMIN = 500
FMAX = 12500
MAX_AUDIO_FILES = 1500

input_dir = 'basic_test/birds/'
output_dir = 'basic_test/melspectrogram_dataset/'
model_dir = 'basic_test/'
soundscape_dir = 'basic_test/'

Hello Kaare
utils


### Get metadata

In [2]:
# Load metadata file
train = pd.read_csv('train_metadata.csv',)

# Limit the number of training samples and classes
# First, only use high quality samples
train = train.query('rating>=4')

# Second, assume that birds with the most training samples are also the most common
# A species needs at least 200 recordings with a rating above 4 to be considered common
birds_count = {}
for bird_species, count in zip(train.primary_label.unique(), 
                               train.groupby('primary_label')['primary_label'].count().values):
    birds_count[bird_species] = count
most_represented_birds = [key for key,value in birds_count.items() if value >= 200] 

TRAIN = train.query('primary_label in @most_represented_birds')
LABELS = sorted(TRAIN.primary_label.unique())

# Let's see how many species and samples we have left
print('NUMBER OF SPECIES IN TRAIN DATA:', len(LABELS))
print('NUMBER OF SAMPLES IN TRAIN DATA:', len(TRAIN))
print('LABELS:', most_represented_birds)

NUMBER OF SPECIES IN TRAIN DATA: 27
NUMBER OF SAMPLES IN TRAIN DATA: 8548
LABELS: ['amerob', 'barswa', 'bewwre', 'blujay', 'bncfly', 'carwre', 'compau', 'comrav', 'comyel', 'eursta', 'gbwwre1', 'grekis', 'houspa', 'houwre', 'mallar3', 'norcar', 'normoc', 'redcro', 'rewbla', 'roahaw', 'rubpep1', 'rucspa1', 'sonspa', 'spotow', 'wbwwre1', 'wesmea', 'yeofly1']


## Create model
### Load base model

In [21]:
baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(INPUT_SHAPE)))
#baseModel = ResNet50(weights="imagenet", include_top=False, input_tensor=Input(shape=(224, 224, 3)))

In [22]:
baseModel.summary()

n[0][0]          
__________________________________________________________________________________________________
conv3_block4_out (Activation)   (None, 6, 16, 512)   0           conv3_block4_add[0][0]           
__________________________________________________________________________________________________
conv4_block1_1_conv (Conv2D)    (None, 3, 8, 256)    131328      conv3_block4_out[0][0]           
__________________________________________________________________________________________________
conv4_block1_1_bn (BatchNormali (None, 3, 8, 256)    1024        conv4_block1_1_conv[0][0]        
__________________________________________________________________________________________________
conv4_block1_1_relu (Activation (None, 3, 8, 256)    0           conv4_block1_1_bn[0][0]          
__________________________________________________________________________________________________
conv4_block1_2_conv (Conv2D)    (None, 3, 8, 256)    590080      conv4_block1_1_relu[0][0] 

### Create a new head

In [23]:
headModel = baseModel.output
headModel = AveragePooling2D(pool_size=(2, 4))(headModel)
#headModel = AveragePooling2D(pool_size=(7, 7))(headModel)
headModel = Flatten(name="flatten")(headModel)
headModel = Dense(256, activation="relu")(headModel)
headModel = Dropout(0.5)(headModel)
headModel = Dense(len(LABELS), activation="softmax")(headModel)

### Add to model

In [24]:
model = Model(inputs=baseModel.input, outputs=headModel)
model.summary()

elu[0][0]        
__________________________________________________________________________________________________
conv4_block1_2_bn (BatchNormali (None, 3, 8, 256)    1024        conv4_block1_2_conv[0][0]        
__________________________________________________________________________________________________
conv4_block1_2_relu (Activation (None, 3, 8, 256)    0           conv4_block1_2_bn[0][0]          
__________________________________________________________________________________________________
conv4_block1_0_conv (Conv2D)    (None, 3, 8, 1024)   525312      conv3_block4_out[0][0]           
__________________________________________________________________________________________________
conv4_block1_3_conv (Conv2D)    (None, 3, 8, 1024)   263168      conv4_block1_2_relu[0][0]        
__________________________________________________________________________________________________
conv4_block1_0_bn (BatchNormali (None, 3, 8, 1024)   4096        conv4_block1_0_conv[0][0] 

In [25]:
model.compile(optimizer=tf.keras.optimizers.Adam(lr=0.001),
              loss=tf.keras.losses.CategoricalCrossentropy(label_smoothing=0.01),
              metrics=['accuracy'])

In [26]:
# Add callbacks to reduce the learning rate if needed, early stopping, and checkpoint saving
callbacks = [tf.keras.callbacks.ReduceLROnPlateau(monitor='val_loss', 
                                                  patience=2, 
                                                  verbose=1, 
                                                  factor=0.5),
             tf.keras.callbacks.EarlyStopping(monitor='val_loss', 
                                              verbose=1,
                                              patience=5),
             tf.keras.callbacks.ModelCheckpoint(filepath=model_dir + 'best_model.h5', 
                                                monitor='val_loss',
                                                verbose=0,
                                                save_best_only=True)]

## Load data

In [3]:
# Shuffle the training data and limit the number of audio files to MAX_AUDIO_FILES
TRAIN = shuffle(TRAIN, random_state=RANDOM_SEED)[:MAX_AUDIO_FILES]

In [4]:
samples = []
with tqdm(total=len(TRAIN)) as pbar:
    for idx, row in TRAIN.iterrows():
        pbar.update(1)
        
        if row.primary_label in most_represented_birds:
            audio_file_path = os.path.join(input_dir, row.primary_label, row.filename)
            samples += utils.get_spectrograms(audio_file_path, row.primary_label, output_dir, SAMPLE_RATE, SIGNAL_LENGTH, SPEC_SHAPE, FMIN, FMAX)
            
TRAIN_SPECS = shuffle(samples, random_state=RANDOM_SEED)
print('SUCCESSFULLY EXTRACTED {} SPECTROGRAMS'.format(len(TRAIN_SPECS)))

 66%|██████▌   | 989/1500 [00:42<00:19, 26.59it/s]c:\Users\Bruger\Documents\Fysik\8. Semester\Applied Machine Learning\AppliedML2021\Final Project\final_exam\utils.py:46: RuntimeWarning: invalid value encountered in true_divide
  mel_spec /= mel_spec.max()
100%|██████████| 1500/1500 [01:07<00:00, 22.12it/s]SUCCESSFULLY EXTRACTED 4157 SPECTROGRAMS



In [32]:
train_specs, train_labels = [], []
with tqdm(total=len(TRAIN_SPECS)) as pbar:
    for path in TRAIN_SPECS:
        pbar.update(1)

        # Open image
        spec = Image.open(path)

        # Convert to numpy array
        spec = np.array(spec, dtype='float32')
        
        # Normalize between 0.0 and 1.0
        # and exclude samples with nan 
        spec -= spec.min()
        spec /= spec.max()
        if not spec.max() == 1.0 or not spec.min() == 0.0:
            continue

        # Add channel axis to 2D array
        spec = np.expand_dims(spec, -1)

        # Add new dimension for batch size
        spec = np.expand_dims(spec, 0)

        # Add to train data
        if len(train_specs) == 0:
            train_specs = spec
        else:
            train_specs = np.vstack((train_specs, spec))

        # Add to label data
        target = np.zeros((len(LABELS)), dtype='float32')
        bird = path.replace(os.sep, '/').split('/')[-2]
        target[LABELS.index(bird)] = 1.0
        if len(train_labels) == 0:
            train_labels = target
        else:
            train_labels = np.vstack((train_labels, target))

  9%|▉         | 372/4157 [00:01<00:23, 159.81it/s]<ipython-input-32-96b737023216>:21: RuntimeWarning: invalid value encountered in true_divide
  spec /= spec.max()
  9%|▉         | 391/4157 [00:01<00:22, 165.24it/s]basic_test/melspectrogram_dataset/rewbla\XC483874_2.png
basic_test/melspectrogram_dataset/bewwre\XC217856_0.png
basic_test/melspectrogram_dataset/houwre\XC513953_1.png
basic_test/melspectrogram_dataset/bewwre\XC553922_2.png
basic_test/melspectrogram_dataset/rewbla\XC135673_0.png
basic_test/melspectrogram_dataset/wbwwre1\XC526140_0.png
basic_test/melspectrogram_dataset/comrav\XC209766_1.png
basic_test/melspectrogram_dataset/houwre\XC590621_0.png
basic_test/melspectrogram_dataset/houwre\XC304858_1.png
 66%|██████▌   | 2733/4157 [00:50<00:26, 53.81it/s]


KeyboardInterrupt: 

### Preprocess

In [ ]:
tf.keras.applications.resnet.preprocess_input

## Train

In [ ]:
# Let's train the model for a few epochs
model.fit(train_specs, 
          train_labels,
          batch_size=32,
          validation_split=0.2,
          callbacks=callbacks,
          epochs=25)